# Just run all definitions

In [15]:
import psycopg2
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as reg
warnings.filterwarnings('ignore')

In [2]:
# EnrichDB
def query(query,password=None):
    if password == None:
        password=str(input('Enter the Database Password: '))
    else:
        password = password
    feeddata = {
    "database":"primary",
    "user":"enrich-dashboard",
    "password": password,
    "host":"analytics.enrich-legacy.groupbycloud.com",
    "port":"5432"
    }
    connection = psycopg2.connect(**feeddata)
    return pd.read_sql(query, connection)

In [3]:
def unique_attributes(customer_id):
    unique_attributes = f'''
    SELECT 
        ga.customer_id, 
        sba.attribute_id, 
        sb.bucket_id 
    
    FROM (
        SELECT 
            customer_id AS "customer_id", 
            g.id AS "guideline_id" 
        
        FROM guidelines g ) AS ga
        INNER JOIN strategy_buckets_attributes sba ON sba.guideline_id = ga."guideline_id"
        INNER JOIN (
        SELECT 
            id, 
            bucket_id FROM strategy_buckets sb 
        ) AS sb ON sb.id = sba.strategy_bucket_id 
        WHERE ga."customer_id" = {customer_id}
        GROUP BY 1,2,3
    '''
    result = query(unique_attributes, '3LT9tsYF7wVHkRjURruW967K')
    return result

In [4]:
def curation(method='uncurated', password=None, _print_supported_values=True, **kwargs):
    '''
    Commonly used queries for DSA work

    Parameters
    ==========
    password: str, password for the SQL database
    group: int, group to return for RegEx searc
    filter: bool, if True, return dataframe with only valid answers
    inplace: bool, if True, curated DataFrame replaces existing DataFrame. False by default.
    '''

    if password == None:
        password=str(input('Enter the Database Password: '))
    else:
        password = password

    keys = [
        'customer_id',
        'attribute_id',
        'snake_case_name',
        'strategy_version_id'
        ]
    # Initialization
    customer_id = None
    attribute_id = None
    snake_case_name = None
    strategy_version_id = None

    for key, val in kwargs.items():
        if key not in keys:
            msg = f"{key!r} is not a valid value for {key}"
            if _print_supported_values:
                msg += f"; supported values are {', '.join(map(repr, keys))}"
            raise ValueError(msg)
        elif key == 'customer_id':
            customer_id = val
        elif key == 'attribute_id':
            attribute_id = val
        elif key == 'snake_case_name':
            snake_case_name = val
        elif key == 'strategy_version_id':
            strategy_version_id = val
        else:
            continue

    feeddata = {
        "database":"primary",
        "user":"enrich-dashboard",
        "password": password,
        "host":"analytics.enrich-legacy.groupbycloud.com",
        "port":"5432"
    }
    connection = psycopg2.connect(**feeddata)

    uncurated_count=f'''
    WITH 
        /*
        * bulk n/a by attribute and bucket
        * 
        */
        active_strategy_bucket_atts AS (
                -- curation_tasks that haven't gone out yet. Includes freetext
                SELECT DISTINCT 
                    b.id AS bucket_id
                    ,A.NAME AS "Attribute"
                    ,a.snake_case_name
                    ,B.NAME AS "Bucket"
                FROM BUCKETS AS B
                JOIN STRATEGY_BUCKETS AS SB ON
                    B.ID = SB.BUCKET_ID
                JOIN STRATEGY_BUCKETS_ATTRIBUTES AS SBA ON
                    SB.ID = SBA.STRATEGY_BUCKET_ID
                JOIN STRATEGY_VERSIONS ON
                    SB.STRATEGY_VERSION_ID = STRATEGY_VERSIONS.ID
                JOIN ATTRIBUTES AS A ON
                    SBA.ATTRIBUTE_ID = A.ID
                WHERE STRATEGY_VERSIONS.CUSTOMER_ID = {customer_id}
                AND SBA.STATUS = 'OPT_IN'
                AND sb.active = True
                AND STRATEGY_VERSIONS.active = True
                AND LOWER(a.name) LIKE LOWER('{snake_case_name}')
                ORDER BY 
                    "Attribute"
                    ,"Bucket"
        ),
        products_from_buckets AS ( 
            SELECT
                bucket
                ,ba.bucket_id
                ,id AS product_id
                ,external_id
                ,name AS product_name
                ,description AS long_desc
                ,url
                ,image_url
                ,cast(products.custom_fields AS jsonb) AS "custom_fields" 
            FROM products
            INNER JOIN(
                SELECT
                    product_id
                    ,bucket_id
                    ,b.name AS bucket
                FROM products_buckets AS pb
                INNER JOIN buckets AS b
                    ON b.id = bucket_id
                WHERE customer_id = {customer_id}
                AND EXISTS (
                    SELECT
                        bucket_id
                    FROM active_strategy_bucket_atts AS asba
                    WHERE asba.bucket_id = pb.bucket_id
                )
            ) AS ba
            ON product_id = products.id
            WHERE customer_id = {customer_id}
            AND active = True
        ),
        not_curated_yet AS (
            SELECT
                *
            FROM products_from_buckets AS pfb
            WHERE NOT EXISTS (
                    SELECT
                        name AS attribute
                        ,external_id
                        ,product_name
                    FROM curated_product_fields AS cpf
                    WHERE LOWER(name) IN (SELECT DISTINCT snake_case_name FROM active_strategy_bucket_atts)
                    AND cpf.customer_id = {customer_id}
                    AND cpf.product_id = pfb.product_id
            )
        )
        SELECT
            bucket_id,
            bucket,
            COUNT(external_id)
        FROM not_curated_yet
        GROUP BY 1,2
        '''

    uncurated=f'''
    WITH 
        /*
        * bulk n/a by attribute and bucket
        * 
        */
        active_strategy_bucket_atts AS (
                -- curation_tasks that haven't gone out yet. Includes freetext
                SELECT DISTINCT 
                    b.id AS bucket_id
                    ,A.NAME AS "Attribute"
                    ,a.snake_case_name
                    ,B.NAME AS "Bucket"
                FROM BUCKETS AS B
                JOIN STRATEGY_BUCKETS AS SB ON
                    B.ID = SB.BUCKET_ID
                JOIN STRATEGY_BUCKETS_ATTRIBUTES AS SBA ON
                    SB.ID = SBA.STRATEGY_BUCKET_ID
                JOIN STRATEGY_VERSIONS ON
                    SB.STRATEGY_VERSION_ID = STRATEGY_VERSIONS.ID
                JOIN ATTRIBUTES AS A ON
                    SBA.ATTRIBUTE_ID = A.ID
                WHERE STRATEGY_VERSIONS.CUSTOMER_ID = {customer_id}
                AND SBA.STATUS = 'OPT_IN'
                AND sb.active = True
                AND STRATEGY_VERSIONS.active = True
                AND LOWER(a.name) LIKE LOWER('{snake_case_name}')
                ORDER BY 
                    "Attribute"
                    ,"Bucket"
        ),
        products_from_buckets AS ( 
            SELECT
                bucket
                ,ba.bucket_id
                ,id AS product_id
                ,external_id
                ,name AS product_name
                ,description AS long_desc
                ,url
                ,image_url
                ,cast(products.custom_fields AS jsonb) AS "custom_fields" 
            FROM products
            INNER JOIN(
                SELECT
                    product_id
                    ,bucket_id
                    ,b.name AS bucket
                FROM products_buckets AS pb
                INNER JOIN buckets AS b
                    ON b.id = bucket_id
                WHERE customer_id = {customer_id}
                AND EXISTS (
                    SELECT
                        bucket_id
                    FROM active_strategy_bucket_atts AS asba
                    WHERE asba.bucket_id = pb.bucket_id
                )
            ) AS ba
            ON product_id = products.id
            WHERE customer_id = {customer_id}
            AND active = True
        ),
        not_curated_yet AS (
            SELECT
                *
            FROM products_from_buckets AS pfb
            WHERE NOT EXISTS (
                    SELECT
                        name AS attribute
                        ,external_id
                        ,product_name
                    FROM curated_product_fields AS cpf
                    WHERE LOWER(name) IN (SELECT DISTINCT snake_case_name FROM active_strategy_bucket_atts)
                    AND cpf.customer_id = {customer_id}
                    AND cpf.product_id = pfb.product_id
            )
        )
        SELECT
            *,
            '{snake_case_name}' AS "attribute",
            'uncurated' AS "value"
        FROM not_curated_yet
        '''

    guideline = f'''
    SELECT g.guideline FROM guidelines g
    INNER JOIN "attributes" a2 ON
        g.attribute_id = a2.id
    WHERE 
        g.customer_id = {customer_id}
    AND a2."snake_case_name" = '{snake_case_name}'
    '''

    strat_buckets = f'''
    SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_id}) AS active_strat
    INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
    WHERE sba.attribute_id = {attribute_id} AND sba.status = 'OPT_IN' 
    '''
    curated = q = f'''
    WITH 
        /*
        * bulk n/a by attribute and bucket
        * 
        */
        active_strategy_bucket_atts AS (
                -- curation_tasks that haven't gone out yet. Includes freetext
                SELECT DISTINCT 
                    b.id AS bucket_id
                    ,A.NAME AS "Attribute"
                    ,a.snake_case_name
                    ,B.NAME AS "Bucket"
                FROM BUCKETS AS B
                JOIN STRATEGY_BUCKETS AS SB ON
                    B.ID = SB.BUCKET_ID
                JOIN STRATEGY_BUCKETS_ATTRIBUTES AS SBA ON
                    SB.ID = SBA.STRATEGY_BUCKET_ID
                JOIN STRATEGY_VERSIONS ON
                    SB.STRATEGY_VERSION_ID = STRATEGY_VERSIONS.ID
                JOIN ATTRIBUTES AS A ON
                    SBA.ATTRIBUTE_ID = A.ID
                WHERE STRATEGY_VERSIONS.CUSTOMER_ID = {customer_id}
                AND SBA.STATUS = 'OPT_IN'
                AND sb.active = True
                AND STRATEGY_VERSIONS.active = True
                AND LOWER(a.name) LIKE LOWER('{snake_case_name}')
                ORDER BY 
                    "Attribute"
                    ,"Bucket"
        ),
        products_from_buckets AS ( 
            SELECT
                bucket
                ,ba.bucket_id
                ,id AS product_id
                ,external_id
                ,name AS product_name
                ,description
                ,url
                ,image_url
            FROM products
            INNER JOIN(
                SELECT
                    product_id
                    ,bucket_id
                    ,b.name AS bucket
                FROM products_buckets AS pb
                INNER JOIN buckets AS b
                    ON b.id = bucket_id
                WHERE customer_id = {customer_id}
                AND EXISTS (
                    SELECT
                        bucket_id
                    FROM active_strategy_bucket_atts AS asba
                    WHERE asba.bucket_id = pb.bucket_id
                )
            ) AS ba
            ON product_id = products.id
            WHERE customer_id = {customer_id}
            AND active = True
        ),
        not_curated_yet AS (
            SELECT
                *
            FROM products_from_buckets AS pfb
            WHERE EXISTS (
                    SELECT
                        name AS attribute
                        ,external_id
                        ,product_name
                    FROM curated_product_fields AS cpf
                    WHERE LOWER(name) IN (SELECT DISTINCT snake_case_name FROM active_strategy_bucket_atts)
                    AND cpf.customer_id = {customer_id}
                    AND cpf.product_id = pfb.product_id
            )
        )
        SELECT
            *,
            '{snake_case_name}' AS "attribute"
        FROM not_curated_yet
        '''
    curated_count = f'''
    WITH 
        /*
        * bulk n/a by attribute and bucket
        * 
        */
        active_strategy_bucket_atts AS (
                -- curation_tasks that haven't gone out yet. Includes freetext
                SELECT DISTINCT 
                    b.id AS bucket_id
                    ,A.NAME AS "Attribute"
                    ,a.snake_case_name
                    ,B.NAME AS "Bucket"
                FROM BUCKETS AS B
                JOIN STRATEGY_BUCKETS AS SB ON
                    B.ID = SB.BUCKET_ID
                JOIN STRATEGY_BUCKETS_ATTRIBUTES AS SBA ON
                    SB.ID = SBA.STRATEGY_BUCKET_ID
                JOIN STRATEGY_VERSIONS ON
                    SB.STRATEGY_VERSION_ID = STRATEGY_VERSIONS.ID
                JOIN ATTRIBUTES AS A ON
                    SBA.ATTRIBUTE_ID = A.ID
                WHERE STRATEGY_VERSIONS.CUSTOMER_ID = {customer_id}
                AND SBA.STATUS = 'OPT_IN'
                AND sb.active = True
                AND STRATEGY_VERSIONS.active = True
                AND LOWER(a.name) LIKE LOWER('{snake_case_name}')
                ORDER BY 
                    "Attribute"
                    ,"Bucket"
        ),
        products_from_buckets AS ( 
            SELECT
                bucket
                ,ba.bucket_id
                ,id AS product_id
                ,external_id
                ,name AS product_name
                ,description
                ,url
                ,image_url
            FROM products
            INNER JOIN(
                SELECT
                    product_id
                    ,bucket_id
                    ,b.name AS bucket
                FROM products_buckets AS pb
                INNER JOIN buckets AS b
                    ON b.id = bucket_id
                WHERE customer_id = {customer_id}
                AND EXISTS (
                    SELECT
                        bucket_id
                    FROM active_strategy_bucket_atts AS asba
                    WHERE asba.bucket_id = pb.bucket_id
                )
            ) AS ba
            ON product_id = products.id
            WHERE customer_id = {customer_id}
            AND active = True
        ),
        not_curated_yet AS (
            SELECT
                *
            FROM products_from_buckets AS pfb
            WHERE EXISTS (
                    SELECT
                        name AS attribute
                        ,external_id
                        ,product_name
                    FROM curated_product_fields AS cpf
                    WHERE LOWER(name) IN (SELECT DISTINCT snake_case_name FROM active_strategy_bucket_atts)
                    AND cpf.customer_id = {customer_id}
                    AND cpf.product_id = pfb.product_id
            )
        )
        SELECT
            bucket_id,
            bucket,
            COUNT(external_id)
        FROM not_curated_yet
        GROUP BY 1,2
        '''
    customers = f'''
    SELECT 
    id,
    name,
    num_top_categories,
    num_categories,
    num_products,
    num_curated_fields,
    (num_curated_fields / num_products) AS "cf_per_p"
    FROM customers WHERE active = TRUE AND num_curated_fields != 0 ORDER BY num_products DESC
    '''
    if method == 'uncurated_count':
        return pd.read_sql(uncurated_count, connection)
    
    elif method == 'uncurated':
        return pd.read_sql(uncurated, connection)

    elif method == 'guideline':
        return pd.read_sql(guideline, connection).iloc[0,0]
    
    elif method == 'strat_buckets':
        return pd.read_sql(strat_buckets, connection)
    
    elif method == 'curated':
        df = pd.read_sql(curated, connection)
        print('Curated SKUs retrieved')
        S = ' OR product_id = '.join([str(x) for x in df['product_id']])
        print('Product IDs retrieved')
        print('Retrieving curated values...')
        a = (query(f'''SELECT product_id, value FROM (
          SELECT product_id, value FROM curated_product_fields 
          WHERE customer_id = {customer_id} AND name = '{snake_case_name}') AS scope
          ''', '3LT9tsYF7wVHkRjURruW967K'))
        print('Curated values retrieved')
        return pd.merge(df,a[['product_id','value']], on = 'product_id')
    
    elif method == 'curated_count':
        return pd.read_sql(curated_count, connection)

    elif method == 'customers':
        return pd.read_sql(customers, connection)

    elif method == 'data':
        customers = pd.read_sql(customers, connection)
        data_list = []
        for index, row in customers.iterrows():
            try:
                dataf = data(str(row.loc['id']))
            except:
                continue
            if dataf.shape[0] == 0:
                continue
            dataf['client'] = row.loc['name']
            data_list.append(dataf)
        return pd.concat(data_list,ignore_index=True)

    else:
        print("method does not exist")    

In [5]:
def data(customer_id,password=None):

    if password == None:
        password=str(input('Enter the Database Password: '))
    else:
        password = password

    ab = unique_attributes(customer_id)
    attribute_ids = list(set(list(ab['attribute_id'])))

    df_list = []
    df_list2 = []

    for id_ in attribute_ids[:]:
        snake = query(f'''SELECT snake_case_name FROM attributes WHERE id = {id_}''', password=password)['snake_case_name'][0]
        df = curation(password=password, method='uncurated_count',customer_id=customer_id, snake_case_name=snake, )
        buckets = list(ab[ab['attribute_id'] == id_]['bucket_id'])
        df = df[df['bucket_id'].isin(buckets)]
        df['curation'] = 'uncurated'
        df['snake_case_name'] = snake
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        df_list.append(df)

    df1 = pd.concat(df_list)
    for id_ in attribute_ids[:]:
        snake = query(f'''SELECT snake_case_name FROM attributes WHERE id = {id_}''', password=password)['snake_case_name'][0]
        df = curation(password=password, method='curated_count',customer_id=customer_id, snake_case_name=snake)
        buckets = list(ab[ab['attribute_id'] == id_]['bucket_id'])
        df = df[df['bucket_id'].isin(buckets)]
        df['curation'] = 'curated'
        df['snake_case_name'] = snake
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        df_list2.append(df)
    df2 = pd.concat(df_list2)
    return pd.concat([df1,df2], ignore_index=True)

# Just run all definitions

In [6]:
lovelyskin = data(customer_id = 115, password = '3LT9tsYF7wVHkRjURruW967K')

In [7]:
lovelyskin

snake_case_name bucket_id                                bucket count  \
0         preferences      7962                    Sun Care & Tanning     6   
1         preferences      7964                    Exfoliants & Masks     2   
2         preferences      7400  Facial Cleansers & Cosmetic Removers     1   
3         preferences      7381                Lotions & Moisturizers     1   
4        product_type      7962                    Sun Care & Tanning     2   
..                ...       ...                                   ...   ...   
546  application_area      7964                    Exfoliants & Masks  1416   
547  application_area      7996           False Eyelashes & Adhesives    18   
548  application_area      8003                            Lip Liners   197   
549  application_area      7998     Foundations & Tinted Moisturizers  1636   
550  application_area      8002         Concealers & Color Correctors   437   

      curation  
0    uncurated  
1    uncurated  
2    uncurated  
3    uncurated  
4    uncurated  
..         ...  
546    curated  
547    curated  
548    curated  
549    curated  
550    curated  

[551 rows x 5 columns]

In [12]:
lovely_skin=pd.read_csv('lovelyskin.csv')

In [16]:
lovely_skin['buckets'].explode().value_counts()[0:500]

Serums & Treatments                                      2507
Lotions & Moisturizers                                   2169
Foundations & Tinted Moisturizers                        1636
Hair Cleaning & Treatments                               1606
Lipsticks, Glosses, & Stains                             1488
Exfoliants & Masks                                       1417
Facial Cleansers & Cosmetic Removers                     1049
Eyeshadow                                                 895
Skin Care Variety Packs                                   866
Nail Polishes & Adornment                                 855
Skin Treatments                                           753
Hair Styling Products                                     732
Makeup Tools & Accessories                                684
Sun Care & Tanning                                        649
Concealers & Color Correctors                             437
Skin Care Tools & Accessories                             428
Lip Care